In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from pyspark.sql import functions as F
from pyspark.sql.types import *

from distribution_cost.configuration import spark_config
from distribution_cost.configuration.app import AppConfig
from distribution_cost.configuration.data import DataConfig
from distribution_cost.infra import oracle
from distribution_cost.domain import kpis

/gpfs/user/e587246/dco00/conf/application.yml
/gpfs/user/e587246/dco00


In [2]:
# Database uri
app_config = AppConfig()
db_uri = app_config.db_uri_jdbc
db_uri

'jdbc:oracle:thin:BRC_E587247/uOR3hLZr@//pyox2k01:1521/BRCEX_PP2'

In [3]:
# Create spark session
spark_context, spark_session = spark_config.get_spark(app_name="app-distribution-cost",
                                                      executors=4, executor_cores=4, executor_mem='4g',
                                                      dynamic_allocation=True, max_executors=8)

In [4]:
xls_2019 =pd.ExcelFile("/gpfs/user/e587247/data/landing/obj/TCD_Budget_2019 MCV_RBCV_Ekimetrics.xlsx")
xls_2018 =pd.ExcelFile("/gpfs/user/e587247/data/landing/obj/TCD_RBCV_-_Budget 2018.xlsx")

In [49]:
dfOBJ_1 = pd.read_excel(xls_2018,'TCD')
dfOBJ_1 = dfOBJ_1.dropna()
dfOBJ_1.columns = dfOBJ_1.iloc[0]
dfOBJ_1 = dfOBJ_1[1:]
dfOBJ_1 = dfOBJ_1.reset_index(drop=True)
dfOBJ_1['Year'] = '2018'
dfOBJ_1['Date'] = '01/'+ dfOBJ_1['mois'].astype(str) + '/' + dfOBJ_1['Year']

for c in dfOBJ_1.columns:
    dfOBJ_1=dfOBJ_1.rename(columns={c : c.strip()})
    
dfOBJ_1 = dfOBJ_1.drop(["PPI unit","Trsm unit","Trfi unit","AF unit","Maco unit","MCV RC130 unit","PPI","Trsm","Trfi","AF","maco","MCV RC130"],axis=1)
dfOBJ_1.columns

dfOBJ_2 = pd.read_excel(xls_2019,'TCD')
dfOBJ_2 = dfOBJ_2.dropna()
dfOBJ_2.columns = dfOBJ_2.iloc[0]
dfOBJ_2 = dfOBJ_2[1:]
dfOBJ_2 = dfOBJ_2.reset_index(drop=True)
dfOBJ_2['Year'] = '2019'
dfOBJ_2['Date'] = '01/'+ dfOBJ_2['mois'].astype(str) + '/' + dfOBJ_2['Year']

for c in dfOBJ_2.columns:
    dfOBJ_2 = dfOBJ_2.rename(columns={c : c.strip()})

dfOBJ_2 = dfOBJ_2.drop(["PVR unit dev","PPI unit","Trsm unit","Trfi unit","AF unit","Maco unit","MCV RC130 unit","PPI","Trsm","Trfi","AF","maco","MCV RC130"],axis=1)
dfOBJ_2.columns

Index(['Marque', 'LIB PAYS', 'mois', 'TYPE', 'LIB SS FAM 2', 'ENERGIE',
       'canal AE', 'volume', 'PVR unit', 'MCV unit', 'RBCV unit',
       'Prime performance unit', 'Promo client final unit',
       'Aides société unit', 'Prime diff unit', 'Buy back unit',
       'taux MCV RC130', 'PVR', 'MCV', 'RBCV', 'RM035', 'RM036', 'RM037',
       'RM039', 'RM280', 'Year', 'Date'],
      dtype='object', name=24)

In [50]:
dfOBJ = pd.concat([dfOBJ_1,dfOBJ_2])

In [51]:
dfOBJ=dfOBJ.drop_duplicates()

In [87]:
dfOBJ['Prime performance unit']=dfOBJ['Prime performance unit'].astype(float)
dfOBJ['Promo client final unit']=dfOBJ['Promo client final unit'].astype(float)
dfOBJ['Aides société unit']=dfOBJ['Aides société unit'].astype(float)
dfOBJ['Prime diff unit']=dfOBJ['Prime diff unit'].astype(float)
dfOBJ['MCV']=dfOBJ['MCV'].astype(float)
dfOBJ['RM035']=dfOBJ['RM035'].astype(float)
dfOBJ['RM039']=dfOBJ['RM039'].astype(float)
dfOBJ['RM037']=dfOBJ['RM037'].astype(float)
dfOBJ['MCV unit']=dfOBJ['MCV unit'].astype(float)
dfOBJ['Buy back unit']=dfOBJ['Buy back unit'].astype(float)
dfOBJ['RM280']=dfOBJ['RM280'].astype(float)
dfOBJ['taux MCV RC130']=dfOBJ['taux MCV RC130'].astype(float)
dfOBJ['RM036']=dfOBJ['RM036'].astype(float)
dfOBJ['PVR unit']=dfOBJ['PVR unit'].astype(float)
dfOBJ['RBCV unit']=dfOBJ['RBCV unit'].astype(float)
dfOBJ['PVR']=dfOBJ['PVR'].astype(float)
dfOBJ['RBCV']=dfOBJ['RBCV'].astype(float)
dfOBJ['volume']=dfOBJ['volume'].astype(float)


In [53]:
column_value_fam={'208 VP+VU':'208',
'PARTNER K9 VU':'PARTNER K9',
'308 T9 BL VP+VU':'308',
'B9 VP':'B9',
'K0 AP VP B':"K0 A¨P",
"PARTNER K9 VP":"PARTNER K9",
"508 R8 BL":"508 R8",
'K0 AP VU':'K0 AP',
'308 SW T9':'308',
"K0 AP VP H":"K0 AP",
"508 R8 SW":"508 R8",
'208 P21 ELEC':'208 P21',
'3008 P84 PHEV':'3008 P84',
'508 R83 PHEV':'508 R8',
'508 R82 PHEV':'508 R8',
'PARTNER B9 VU ELECTRIQUE':'PARTNER K9',
'2008 P24 ELEC':'2008 P24 ELEC',
'K0 AP VU ELEC':'K0 AP',
'B9 VU':'B9',
'208 P21 VU':'208 P21',
'K0 AP VP ELEC':'K0 AP',
'BIPPER VU':'BIPPER',
'K0 AP VP H CKD':'K0 AP',
'3008 P84 CKD NAM':'3008 P84',
'5008 P87 CKD NAM':'5008 P87',
'K0 AP VP RUSSIE':'K0 AP RUSSIE',
'K0 AP VU RUSSIE':'K0 AP RUSSIE',
'PARTNER VP':'PARTNER K9',
'PARTNER VU':'PARTNER K10',
'508 BERLINE TH':'508',
'3008TH':'3008 P84'}

dfOBJ['LIB SS FAM 2'] = dfOBJ['LIB SS FAM 2'].replace(column_value_fam)

In [45]:
dfOBJ['Prime performance unit'].astype

<bound method NDFrame.astype of 0          0.000000
1        470.706646
2        320.296936
3          0.000000
4        532.385447
            ...    
52712    200.000000
52713    600.000000
52714    200.000000
52715    200.000000
52716    600.000000
Name: Prime performance unit, Length: 53861, dtype: float64>

In [54]:
column_value_typutil={'VD/VA':'Demo vehicles',
'VD/VS':'Demo vehicles','PARTICULIERS':'B2C','AUTRES SOCIETES':'B2B - Adm','GRANDS COMPTES':'B2B - Adm','LCD DIRECT':'Renters','LCD RESEAU':'Renters','PERSONNEL':'B2C'}

dfOBJ['canal AE'] = dfOBJ['canal AE'].replace(column_value_typutil)

In [55]:
column_value_typveh={'VP':'PC',
'VU':'CV'}

dfOBJ['TYPE'] = dfOBJ['TYPE'].replace(column_value_typveh)

## Load madax on oracle

In [89]:
#dfOBJ_spark = spark_session.createDataFrame(dfOBJ)
dfOBJ_spark.write.jdbc(url=db_uri, table="SMKT015_REF_OBJ", mode="overwrite")

In [58]:
#dfPROMO.write.mode("overwrite").partitionBy("Country").parquet("hdfs:///user/e587247/data/refined/ope/")